# Import & Functions

In [ ]:
import os
import sys
import time
import glob
import numpy as np
import pandas as pd

import nibabel as nib
import SimpleITK as sitk

from scipy import ndimage
import torch
from util.spatialTransform import SpatialTransformer
from util.util import save_as_3D, create_4DCT

In [ ]:
def resize_dvf(input_f, resize_value=[128,128,128], out_type = 'int16'):
    """
    Resize 3D data to fit the same size
    
    :param input_f: nifti file (.nii.gz
	:param x: image x,y,z dimensions
	:param y: image y dimensions
	:param z: image z dimensions
    :param resize_value: image dims wanted, by default 256
	:return: nothing but create .nii.gz file (type: float32)
    
    """
    # Load nifti image
    img = nib.load(input_f)
    hdr = img.header
    # print("Resize file", input_f)
    # print("Original image shape: ",img.shape)
    
    # Get ndarray
    data = img.get_fdata()
    current_width = data.shape[0]
    current_height = data.shape[1]
    current_depth = data.shape[2]

    width = current_width / resize_value[0]
    height = current_height / resize_value[1]
    depth = current_depth / resize_value[2]
    
    # Calcul zoom ratio
    width_factor = 1 / width
    height_factor = 1 / height
    depth_factor = 1 / depth
    
    new_img = ndimage.zoom(data, zoom=(width_factor, height_factor, depth_factor, 1), order=1)
    
    # Create new image    
    # print("New image shape (after zoom): ", new_img.shape)
    if out_type == 'float32':
        new_img = new_img.astype(np.float32)
    if out_type == 'int16':
        new_img = new_img.astype(np.int16)
    if out_type == 'uint8':
        new_img = new_img.astype(np.uint8)
    
    return new_img

In [ ]:
def mkdir(dirpath):
    """
    Create directory if no directory with the given path
    :param dirpath: directory path
    :return: nothing but create directory (if needed)
    """
    if not os.path.exists(dirpath):
        print("Creating directory at:", dirpath)
        os.makedirs(dirpath)

# Postprocessing

## Initialisation

In [ ]:
# 0. Init
csv_dir      = './datasets/000_csv/'
original_dir = './datasets/001_original/'
bb_dir       = './datasets/002_bounding_box/'

result_dir = './results/pretrained_model/'
bb_values = pd.read_csv('./datasets/000_csv/final_bb.csv')

## Choose used input file (128x128x128)

In [ ]:
# 1. Choose input file (128x128x128)
# Write path of input file
INPUT_FILE = './datasets/imagesTs/LungCT_0100_0005.nii.gz'

# 2. Find case
name = os.path.basename(INPUT_FILE)
case = name.split('_')[1]

## Create output directory

In [ ]:
# 3. Create Directories of the case
generated_dvf_dir = os.path.join(result_dir, case, 'dvf/')
warped_dir = os.path.join(result_dir, case, 'warped_initialsize/')

# Create directories
mkdir(warped_dir)

## To initial size

In [ ]:
# 4. Get BB size
bb_file = os.path.join(bb_dir,name)
img = nib.load(bb_file)
dim_X, dim_Y, dim_Z = img.shape
resize_values = [dim_X, dim_Y, dim_Z]

# 5. Get DVF files
dvf_files = sorted(glob.glob(generated_dvf_dir + "/*"))

# Find original input file & read bounding box from preprocessing
original_file = os.path.join(original_dir, name)

for dvf_file in dvf_files[:]:
    start_time = time.time()
    
    # 6. DVF resize to bb size
    print('--- Start resampled to BB size')
    bb_arr = resize_dvf(dvf_file, resize_values, 'float32')
    
    # 7. Get bounding box position
    tmp_values = bb_values[bb_values.case == int(case)].values[0][2:]
    
    # 8. Put BB into initial size
    print('--- Start to insert back to initial size')
    input_arr = nib.load(original_file).get_fdata()
    D,W,H = input_arr.shape
    result = np.zeros((D, W, H, 3))
    result[tmp_values[0]:tmp_values[0]+tmp_values[3], tmp_values[1]:tmp_values[1]+tmp_values[4], tmp_values[2]:tmp_values[2]+tmp_values[5], :] = bb_arr

    # 9. Warp image with DVF
    print('--- Start warping')
    input_field = result.transpose(3,0,1,2)
    
    # Apply same transform as in model
    transform = SpatialTransformer(np.asarray(input_arr.shape))

    # to torch
    input_field_t = torch.from_numpy(input_field.astype(np.float32)).unsqueeze(0)
    moving_t = torch.from_numpy(input_arr.astype(np.float32)).unsqueeze(0).unsqueeze(1)

    warped = transform(moving_t, input_field_t)
    warped_file = os.path.join(warped_dir, os.path.basename(dvf_file).replace('-dvf','-warped'))
    save_as_3D(warped, [original_file], warped_file, range_intensity=[-1,1])
    print("--- Finish file : ", dvf_file)
    print("--- %s seconds ---" % (time.time() - start_time))

## Generate 4DCT

In [ ]:
# 10. Create the associate 4DCT
final_file = result_dir + name.rsplit('_',1)[0] + '_4DCT_postprocess.nii.gz'
create_4DCT([original_file], warped_dir, final_file, loop=0)